# 스트림 처리의 기초
- 스파크는 스트림 처리를 위한 다음 두 가지 API를 도입함
  - DStream(Discretized Stream)
  - 구조적 스트리밍
  
- DStream
  - 자바나 파이썬 객체에 대한 <strong>저수준 연산</strong>만 지원
  - 따라서 고수준 최적화 기법을 활용하는데 한계가 있음
  
- 구조적 스트리밍
  - <strong>DataFrame, Dataset</strong> 기반
  - RDD와 DataFrame의 관계처럼 DStream의 주요 기능에 대한 상위 기능을 제공함

## 스트림 처리란
- 스트림 처리(stream processing)는 <strong>신규 데이터를 끊임없이 처리</strong>해 결과를 만들어내는 행위
  - <strong>입력 데이터는 무한</strong>
  - 시작과 끝을 사전에 정의하지 않음
  - 입력데이터의 예시로, 신용카드 전표 정보, 웹사이트 클릭, IoT장비의 센서 데이터 등이 있음
  - 반면, 배치처리는 입력데이터 크기가 미리 정해져있고, 결과를 한 번만 만들어냄

- 스트리밍 애플리케이션은 <strong>이벤트 스트림이 도착한 후 </strong>다양한 쿼리 연산을 수행함
  - ex) 이벤트 타입별 실행 카운트 추적, 시간별로 이벤트의 타입별 실행 카운트 집계 등

- 스트리밍 애플리케이션은 실행하면서 <strong>다양한 버전의 결과를 출력</strong>하거나 키-값 저장소 같은 <strong>외부 'sink' 시스템에 최신 데이터를 저장</strong>할 수도 있음

### 배치 처리와 스트림 처리를 함께 사용하는 경우
- <strong>스트림 입력 데이터를 배치 작업에서 주기적으로 만들어내는 데이터셋과 조인</strong>하는 경우
- <strong>스트리밍 작업의 출력이 배치 작업용 쿼리에 필요</strong>한 경우
- 주의: 모든 비즈니스 로직은 스트리밍과 배치 연산에서 <strong>일관성</strong> 있게 동작해야함
  - 구조적 스트리밍은 배치 애플리케이션 뿐만 아니라 나머지 컴포넌트와 <strong>쉽게 연동</strong> 가능
  - 연속형 애플리케이션: 스트리밍, 배치, 대화형 작업으로 구성된 통합 애플리케이션

### 스트림 처리 사례

#### 통보와 알림
- 실시간으로 특정 이벤트나 이벤트의 패턴을 탐지했을 때 알림

#### 실시간 리포트
- 실시간 대시보드를 만들 때
  - ex) 플랫폼 전체 사용량, 시스템 부하, 실행 시간, 신규 기능 사용량 등을 관찰하기 위한 대시보드

#### 증분형 ETL
- 데이터 웨어하우스에서 정보를 얻는 시간을 줄이기 위해

#### 실시간 제공용 데이터 갱신
- 다른 애플리케이션의 서비스용 데이터를 만들기 위해
  - ex) 웹 분석 서비스: 각 페이지의 방문자 수를 연속적으로 추적하여 최신 방문자 수를 갱신할 때 스트리밍 시스템 사용

#### 실시간 의사결정
- 실시간으로 신규 입력을 분석하고 자동으로 비즈니스 로직에 따라 처리하기 위해
  - ex) 신용카드 고객의 최근 이력을 기준으로 부정행위 여부 자동 판단

#### 온라인 머신러닝
- 실시간 데이터와 이력 데이터를 조합하여 모델 학습

### 스트림 처리의 장점
- <strong>대기 시간이 짧음</strong>
  - 빨리 응답해야하는 애플리케이션이면 <strong>상태를 메모리에 저장</strong>하는 스트리밍 시스템이 필요
  
- <strong>결과 수정 용이</strong>
  - ex) 24시간의 웹 트래픽 통계를 계산하고싶음
    - <strong>배치 처리:</strong> 실행할 때마다 전체 데이터를 <strong>읽고 매번 24시간치 데이터를 계산함</strong>
    - <strong>스트림 처리:</strong> 이전 연산의 상태를 기억하고 <strong>신규 데이터만 계산함</strong>

### 스트림 처리의 과제
- <strong>순서가 뒤섞인 데이터</strong> 처리하기(순서가 중요한 경우)
- <strong>대규모 상태 정보 유지</strong>하기
- <strong>높은 데이터 처리량</strong> 보장하기
- 장애 상황에서도 <strong>정확히 한 번 처리</strong>하기
- <strong>부하 불균형과 뒤처진 서버</strong> 다루기
- 이벤트에 <strong>빠르게 응답</strong>하기
- 다른 저장소 시스템의 <strong>외부 데이터와 조인</strong>하기
- 신규 이벤트 도착 시 출력 싱크의 <strong>갱신 방법 결정</strong>하기
- 출력 시스템에 데이터 저장 시 <strong>트랜잭션 보장</strong>하기
- <strong>런타임에 비즈니스 로직 변경</strong>하기

## 스트림 처리의 핵심 설계 개념
- 애플리케이션이 데이터 처리와 관련된 완벽한 제어권을 가져야하는 분야에선 특히 중요한 부분임
- 스트리밍 API를 설계하는 가장 간단한 방법은 각 이벤트를 애플리케이션에 전달하고 사용자 코드에 반응하도록 만드는 것

### 레코드 단위 처리와 선언형 API

#### 레코드 단위 처리 API를 사용하는 스트리밍 시스템
- 애플리케이션 내부에서 여러 처리 <strong>파이프라인을 연결하는 기능만 제공
- 상태 관리에 어려움이 있음
  - 사용자가 <strong>직접 상태를 추적</strong>해야함
  - 메모리 확보를 위한 상태 제거와 중복 처리 방지 등의 <strong>기능을 직접 구현</strong>해야함
- 이런 시스템을 정교하게 개발하는 것은 어려움(저수준 API를 개발하고 유지 보수하려면 높은 숙련도 필요)

#### 선언형 API를 사용하는 스트리밍 시스템
- 해당 시스템은 애플리케이션을 정의할 때 어떻게 처리할지 대신 <strong>무엇을 처리할지</strong> 지정함
  - 내부적으로 각 연산자의 데이터 처리량과 연산 관련 상태 정보를 <strong>자동으로 추적하고 관련 상태를 신뢰도 있게 저장</strong>함
  - 또한 필요한 경우 장애 지점부터 연산 복구
- <strong>DStream API는 맵, 리듀스, 필터 같은 연산을 기반으로 하는 함수형 API</strong> 제공 
- <strong>구조적 스트리밍은 별도의 프로그래밍 없이 함수형 연산을 훨씬 효율적으로 처리</strong>할 수 있는 SQL형태의 관계형 연산으로 변환함

### 이벤트 시간과 처리 시간

#### 이벤트 시간을 지원하는 시스템
- <strong>원천 시스템에서 각 레코드에 기록한 타임스탬프 기반</strong> 데이터 처리
- 모바일 기기나 IoT 장치 같이 <strong>전송에 지연이 발생할 수 있는 데이터를 수집하는 경우</strong>, 이벤트 시간 처리가 매우 중요
- 하지만 몇 가지 문제점을 고려해야함
  - 늦게 도착한 이벤트를 처리할 수 있도록 <strong>상태를 추적</strong>해야함
  - <strong>이벤트 시간이 속한 특정 시간 윈도우</strong>의 결과를 가장 적절하게 출력할 수 있는 <strong>시점</strong>을 결정해야함
    - <strong>모든 입력 데이터를 수신</strong>했을 가능성이 가장 높은 <strong>시기가 언제</strong>인지 결정해야함

#### 처리 시간을 지원하는 시스템
- <strong>스트리밍 애플리케이션에 레코드가 도착한 시간 기반</strong> 데이터 처리

### 연속형 처리와 마이크로 배치 처리

#### 연속형 처리 기반 시스템
- 각 노드는 <strong>다른 노드에서 전송하는 메시지를 끊임없이 수신</strong>하고 <strong>새로 갱신된 정보를 자신의 하위 노드로 전송</strong>함
- 신규 메시지에 즉시 반응하기 때문에 <strong>전체 입력량이 비교적 적을 때 가장 빠른 응답</strong> 가능
- <strong>레코드 단위 부하</strong>가 크기 때문에 <strong>최대 처리량은 적음</strong>
  - ex) 다음 처리 노드로 메시지 패킷을 보내기 위해 OS를 호출하는 연산 부하가 발생함

#### 마이크로 배치 처리 기반 시스템
- 입력 데이터를 <strong>작은 배치로 모으기 위해 대기</strong>함
  - 여기서 지연 시간이 발생하긴함
  - ex) 500ms 정도
- 배치 잡 실행 방식과 유사하게 <strong>다수의 분산 태스크를 이용해 각 배치를 병렬로 처리함</strong>
  - <strong>배치 시스템의 최적화</strong> 기법 사용 가능
- <strong>연속형에 비해 높은 노드당 처리량</strong>을 얻을 수 있음
  - 더 적은 노드로 같은 양의 데이터 처리 가능
  - 추가적인 레코드별 부하가 없음
- <strong>워크로드 변화에 대응</strong>할 수 있도록 태스크 수를 늘리거나 줄이는 방식인 <strong>부하 분산</strong> 기술을 동적으로 사용 가능

## 스파크의 스트리밍 API

### DStream API
- 2016년까진 가장 널리 사용된 스트림 처리 엔진
- 하지만 몇 가지 제약사항이 있음
  - 많은 장점을 제공하는 구조적 테이블의 개념의 DataFrame이나 Dataset과 달리 <strong>자바나 파이썬의 객체와 함수에 매우 의존적</strong>
    - 최적화 기법을 적용하지 못함
  - 기본적으로 <strong>처리 시간을 기준</strong>으로 동작함
    - 이벤트 시간 기준으로 처리하려면 자체적으로 구현해야함
  - <strong>마이크로 배치 형태</strong>로만 동작함

### 구조적 스트리밍
- 스파크의 구조적 API를 기반으로 하는 고수준 스트리밍 API
  - 스칼라, 자바, 파이썬, R, SQL을 사용해 <strong>구조적 처리를 할 수 있는 모든 환경에서 사용 가능</strong>
  - DStream보다 <strong>더 많은 종류의 최적화</strong> 수행 가능
- <strong>이벤트 시간을 기준</strong>으로 동작함
- 2.2버전까진 마이크로 배치 모드만 지원했지만 2.3부턴 <strong>연속형 처리 모드 지원</strong>